In [1]:
import pandas as pd
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5"]
column_labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5", "Ball Delta", "Ball Y Delta", "LS Delta", "LS Y Delta",
    "Ball 1 Repeat Last 5", "Ball 2 Repeat Last 5", "Ball 3 Repeat Last 5", "Ball 4 Repeat Last 5", "Ball 5 Repeat Last 5", "Lucky Star 1 Repeat Last 5", "Lucky Star 2 Repeat Last 5",
    "Ball 1 Repeat Last 10", "Ball 2 Repeat Last 10", "Ball 3 Repeat Last 10", "Ball 4 Repeat Last 10", "Ball 5 Repeat Last 10", "Lucky Star 1 Repeat Last 10", "Lucky Star 2 Repeat Last 10",
    "Ball 1 Repeat Last 20", "Ball 2 Repeat Last 20", "Ball 3 Repeat Last 20", "Ball 4 Repeat Last 20", "Ball 5 Repeat Last 20", "Lucky Star 1 Repeat Last 20", "Lucky Star 2 Repeat Last 20",
     "Digit 0 Freq", "Digit 1 Freq", "Digit 2 Freq", "Digit 3 Freq", "Digit 4 Freq", "Digit 5 Freq", "Digit 6 Freq", "Digit 7 Freq", "Digit 8 Freq", "Digit 9 Freq"
]
column_indices = {name: i for i, name in enumerate(column_labels)}
digit_indices = {}


df1 = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions", header=0)
df2 = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions digit analysis", header=0)

pd.concat([df1, df2], axis=1)[column_labels].to_csv("euromillions-dataset.csv", index=None)

Num GPUs Available:  0


In [2]:
df = pd.read_csv('euromillions-dataset.csv')
df

,Lucky Star 1,Lucky Star 2,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5,Ball Delta,Ball Y Delta,LS Delta,...,Digit 0 Freq,Digit 1 Freq,Digit 2 Freq,Digit 3 Freq,Digit 4 Freq,Digit 5 Freq,Digit 6 Freq,Digit 7 Freq,Digit 8 Freq,Digit 9 Freq
0,4,12,12,30,38,40,41,4.956057,2.280351,8,...,2,3,2,2,3,0,0,0,1,0
1,7,12,10,29,43,46,49,5.994789,8.282512,5,...,1,2,2,1,3,0,1,1,0,2
2,5,6,1,8,13,29,47,6.393356,3.065942,1,...,0,2,1,1,1,1,1,1,1,1
3,6,12,6,9,25,37,46,5.533986,4.841487,6,...,0,1,2,1,1,1,3,1,0,1
4,2,9,9,19,44,47,50,6.814507,5.407402,7,...,1,1,1,0,3,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,6,9,1,19,36,38,49,6.791539,2.814249,3,...,0,2,0,2,1,0,2,0,1,3
339,9,10,10,25,29,34,45,4.918079,3.143247,1,...,2,2,2,1,2,2,0,0,0,2
340,2,4,6,16,18,39,47,6.169481,3.831449,2,...,0,2,1,1,2,0,2,1,1,1
341,7,12,5,14,35,36,39,5.766281,4.422669,5,...,0,2,1,3,1,2,1,1,0,1


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# --- 1. Data Preparation ---

def create_sequences(data, seq_length, pred_cols):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)].values[::-1]
        y = data.iloc[i + seq_length, :pred_cols].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# --- 2. Load and Preprocess Data ---

# Parameters
seq_length = 7  # Number of past rows to look at
pred_cols = 42  # Number of columns to predict
ball_cols = 7 # Numbers of columns that represent ball numbers
all_cols = 42
random_seed_length = 7 # Length of random seed

In [4]:
max_values = np.concatenate(([12, 12, 50, 50, 50, 50, 50], np.zeros(all_cols - ball_cols)))
min_values = np.concatenate(([1, 1, 1, 1, 1, 1, 1], np.zeros(all_cols - ball_cols)))

# Scale the data
scaler = MinMaxScaler()
scaled_df = df[::-1]
scaled_df = pd.concat([scaled_df, pd.DataFrame([max_values], columns=df.columns)], ignore_index=True)
scaled_df = pd.concat([scaled_df, pd.DataFrame([min_values], columns=df.columns)], ignore_index=True)
scaled_df.shape

(345, 42)

In [5]:
# Scale the data
scaled_df = pd.DataFrame(scaler.fit_transform(scaled_df), columns=df.columns)
scaled_df = scaled_df.iloc[:-2]

# Create sequences
X, y = create_sequences(scaled_df, seq_length, pred_cols)

# Split into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

# Input for the sequence
sequence_input = tf.keras.layers.Input(shape=(seq_length, all_cols))

# Input for the random seed
random_seed_input = tf.keras.layers.Input(shape=(random_seed_length,))

scaled_df.shape

(343, 42)

In [6]:
scaler.inverse_transform([np.concatenate((y[-1], np.zeros(all_cols - pred_cols)))])

array([[ 4.        , 12.        , 12.        , 30.        , 38.        ,
        40.        , 41.        ,  4.9560569 ,  2.28035085,  8.        ,
         2.12132034,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  2.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  2.        ,  1.        ,  4.        ,
         0.        ,  3.        ,  1.        ,  0.        ,  3.        ,
         2.        ,  7.        ,  2.        ,  3.        ,  2.        ,
         2.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ]])

In [7]:
scaler.inverse_transform(X[-1])

array([[ 7.        , 12.        , 10.        , 29.        , 43.        ,
        46.        , 49.        ,  5.9947894 ,  8.2825117 ,  5.        ,
         4.47213595,  0.        ,  2.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  2.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  3.        ,
         2.        ,  2.        ,  0.        ,  4.        ,  5.        ,
         2.        ,  6.        ,  1.        ,  2.        ,  2.        ,
         1.        ,  3.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  2.        ],
       [ 5.        ,  6.        ,  1.        ,  8.        , 13.        ,
        29.        , 47.        ,  6.39335593,  3.06594194,  1.        ,
         4.30116263,  0.        ,  1.        ,  0.        ,  1.        ,
         3.        ,  1.        ,  1.        ,  0.        ,  1.        ,
         1.        ,  1.        ,  4.        ,  1.        ,  2.        ,
         1.     

In [8]:
# for testing sequences
# df = df[::-1]
# print(y[-1])
# print(df.iloc[-1].values[:7])
ball_1_mean = df['Ball 1'].mean()
ball_1_std = df['Ball 1'].std()
print(df['Ball 1'].mean(), df['Ball 1'].std())

ball_2_mean = df['Ball 2'].mean()
ball_2_std = df['Ball 2'].std()
ball_3_mean = df['Ball 3'].mean()
ball_3_std = df['Ball 3'].std()
ball_4_mean = df['Ball 4'].mean()
ball_4_std = df['Ball 4'].std()
ball_5_mean = df['Ball 5'].mean()
ball_5_std = df['Ball 5'].std()

ls_1_mean = df['Lucky Star 1'].mean()
ls_1_std = df['Lucky Star 1'].std()
print(df['Lucky Star 1'].mean(), df['Lucky Star 1'].std())
ls_2_mean = df['Lucky Star 2'].mean()
ls_2_std = df['Lucky Star 2'].std()

# generate random integer from normal distribution
print([
    round(np.random.normal(loc=ls_1_mean, scale=ls_1_std)) / 12.0,
    round(np.random.normal(loc=ls_2_mean, scale=ls_2_std)) / 12.0,
    round(np.random.normal(loc=ball_1_mean, scale=ball_1_std)) / 50.0,
    round(np.random.normal(loc=ball_2_mean, scale=ball_2_std)) / 50.0,
    round(np.random.normal(loc=ball_3_mean, scale=ball_3_std)) / 50.0,
    round(np.random.normal(loc=ball_4_mean, scale=ball_4_std)) / 50.0,
    round(np.random.normal(loc=ball_5_mean, scale=ball_5_std)) / 50.0,
])


9.25072886297376 6.5526293365023625
4.297376093294461 2.6322714039599258
[0.25, 0.5833333333333334, 0.3, 0.22, 0.54, 0.7, 0.76]


In [9]:
# --- 3. Model Definition ---

# LSTM layer
lstm1_1 = tf.keras.layers.LSTM(128, return_sequences=False)(sequence_input)

dense_1 = tf.keras.layers.Dense(128, activation='sigmoid')(lstm1_1)

dense_2 = tf.keras.layers.Dense(35, activation='sigmoid')(dense_1)

concat = tf.keras.layers.Concatenate()([random_seed_input, dense_2])

output = tf.keras.layers.Dense(pred_cols)(concat)

# Create the model with two inputs
lstm_model = tf.keras.models.Model(inputs=[sequence_input, random_seed_input], outputs=output)

In [10]:

# --- 4. Loss Function with Emphasis ---

def custom_loss(y_true, y_pred, random_target):
    # Calculate loss
    squared_error = tf.square(y_true - y_pred)
    weighted_mse_loss = tf.reduce_mean(squared_error)

    # MSE between prediction and random target
    mse_loss_random = tf.reduce_mean(tf.square(y_pred[:, :ball_cols] - random_target))

    # Combine the two losses (you can adjust the weights)
    return tf.abs(0.2 * weighted_mse_loss - mse_loss_random)

def generate_random_target(batch_size):
    targets = []
    for _ in range(batch_size):
        target = np.array([
            round(np.random.normal(loc=ls_1_mean, scale=ls_1_std)) / 12.0,
            round(np.random.normal(loc=ls_2_mean, scale=ls_2_std)) / 12.0,
            round(np.random.normal(loc=ball_1_mean, scale=ball_1_std)) / 50.0,
            round(np.random.normal(loc=ball_2_mean, scale=ball_2_std)) / 50.0,
            round(np.random.normal(loc=ball_3_mean, scale=ball_3_std)) / 50.0,
            round(np.random.normal(loc=ball_4_mean, scale=ball_4_std)) / 50.0,
            round(np.random.normal(loc=ball_5_mean, scale=ball_5_std)) / 50.0,
        ])

        targets.append(target)
    return np.array(targets)

# --- 5. Compile or Load the Model ---
# lstm_model = tf.keras.models.load_model('lstm_model', custom_objects={'custom_loss': custom_loss})
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss)


In [11]:
print(X_train.shape)

print(scaler.data_min_)

print(scaler.data_max_)

(302, 7, 42)
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[12.         12.         50.         50.         50.         50.
 50.          8.70344759 11.56719499 11.          9.05538514  2.
  3.          4.          3.          3.          3.          4.
  7.          5.          6.          6.          5.          9.
  8.          9.          8.          8.          9.          8.
 11.         12.          4.          7.          6.          6.
  6.          3.          4.          3.          4.          3.        ]


In [12]:
# --- 6. Train the Model ---

# Custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

epochs = 50
batch_size = 16

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(0, X_train.shape[0], batch_size):
        # Get batch data

        X_batch = X_train[i:(i + batch_size)]
        y_batch = y_train[i:(i + batch_size)]
        random_seeds_batch = generate_random_target(X_batch.shape[0])

        with tf.GradientTape() as tape:
            # Get predictions
            predictions = lstm_model([X_batch, random_seeds_batch], training=True)

            # Combine the two losses (you can adjust the weights)
            loss = custom_loss(y_batch, predictions, random_seeds_batch)

        # Compute and apply gradients
        gradients = tape.gradient(loss, lstm_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, lstm_model.trainable_variables))

        epoch_loss += loss

    # Print epoch loss
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / (X_train.shape[0] // batch_size)}")



Epoch 1/50, Loss: 0.08296529948711395
Epoch 2/50, Loss: 0.01642872765660286
Epoch 3/50, Loss: 0.00647621788084507
Epoch 4/50, Loss: 0.012049187906086445
Epoch 5/50, Loss: 0.011943481862545013
Epoch 6/50, Loss: 0.008371611125767231
Epoch 7/50, Loss: 0.009155920706689358
Epoch 8/50, Loss: 0.008093351498246193
Epoch 9/50, Loss: 0.011169668287038803
Epoch 10/50, Loss: 0.008423618040978909
Epoch 11/50, Loss: 0.005462139844894409
Epoch 12/50, Loss: 0.006838404573500156
Epoch 13/50, Loss: 0.008031811565160751
Epoch 14/50, Loss: 0.005312307737767696
Epoch 15/50, Loss: 0.006258400157094002
Epoch 16/50, Loss: 0.0054772705771028996
Epoch 17/50, Loss: 0.007120301481336355
Epoch 18/50, Loss: 0.007883693091571331
Epoch 19/50, Loss: 0.006319150328636169
Epoch 20/50, Loss: 0.004574374295771122
Epoch 21/50, Loss: 0.0039363037794828415
Epoch 22/50, Loss: 0.003927580546587706
Epoch 23/50, Loss: 0.004714336711913347
Epoch 24/50, Loss: 0.0041559371165931225
Epoch 25/50, Loss: 0.008217169903218746
Epoch 26/

In [13]:
# --- 8. Evaluate the Model ---

val_loss = 0
for i in range(0, X_val.shape[0], batch_size):
    X_batch = X_val[i:i + batch_size]
    y_batch = y_val[i:i + batch_size]
    random_target_batch = generate_random_target(X_batch.shape[0])

    predictions = lstm_model([X_batch, random_target_batch], training=False)
    loss = custom_loss(y_batch, predictions, random_target_batch)
    val_loss += loss

print(f"Validation Loss: {val_loss / (X_val.shape[0] // batch_size)}")

Validation Loss: 0.0058272285386919975


In [20]:
# --- 9. Make Predictions ---

# Generate a random seed for prediction
prediction_random_seed = generate_random_target(1)

# Get the last sequence from X_val and reshape it for prediction
last_sequence = X[-1].reshape(1, seq_length, all_cols)

# Make a prediction using the last sequence and the random seed
prediction = lstm_model.predict([last_sequence, prediction_random_seed])

# Inverse transform to get predictions in the original scale
prediction = scaler.inverse_transform(np.concatenate((prediction, np.zeros((prediction.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


array([[ 5.35094491e+00,  7.28547853e+00,  7.83691466e+00,
         1.72414187e+01,  3.78795328e+01,  3.40312992e+01,
         4.66652099e+01, -1.80435081e+00,  4.20511490e+00,
         3.00287873e+00, -4.36562906e-01,  7.16337740e-01,
         6.52241886e-01,  3.90592158e-01, -6.30126759e-01,
        -1.55954200e+00,  1.42293754e+00,  2.71010303e+00,
         2.65222400e+00,  3.43855396e-01,  2.96864323e-02,
         2.76870167e+00,  3.78742546e+00,  1.83795838e+00,
        -4.16638708e+00,  3.27170357e+00,  4.11296940e+00,
         5.08702874e-01,  4.53739107e+00,  4.01172304e+00,
         1.84350155e+00,  6.43991876e+00,  5.84357142e-01,
        -4.26963502e+00,  1.77697331e+00, -4.17927718e+00,
        -1.64680266e+00, -4.35826957e-01,  6.76932812e-01,
         3.11439224e-01,  2.09475398e+00,  4.86071840e-01]])

In [21]:
print(scaler.inverse_transform (X[-1])[:, :pred_cols]);

[[ 7.         12.         10.         29.         43.         46.
  49.          5.9947894   8.2825117   5.          4.47213595  0.
   2.          0.          1.          0.          0.          1.
   0.          2.          0.          1.          1.          0.
   3.          2.          2.          0.          4.          5.
   2.          6.          1.          2.          2.          1.
   3.          0.          1.          1.          0.          2.        ]
 [ 5.          6.          1.          8.         13.         29.
  47.          6.39335593  3.06594194  1.          4.30116263  0.
   1.          0.          1.          3.          1.          1.
   0.          1.          1.          1.          4.          1.
   2.          1.          2.          2.          1.          5.
   3.          3.          0.          2.          1.          1.
   1.          1.          1.          1.          1.          1.        ]
 [ 6.         12.          6.          9.         25.     

In [ ]:
# 1. Save as TensorFlow SavedModel (recommended)
lstm_model.save('lstm_model') # Saves to a directory named 'my_full_model'